In [ ]:
# classify the Size_Categorie using SVM

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

In [3]:
# SVM lib
from sklearn import svm
from sklearn.svm import SVC

In [4]:
# split the data

from sklearn.model_selection import train_test_split

# Gridsearch
from sklearn.model_selection import GridSearchCV

In [6]:
# metrics

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
from sklearn.preprocessing import LabelEncoder


## Preprocessing the Data

In [7]:
forest = pd.read_csv('forestfires.csv')

In [8]:
forest.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small


In [9]:
forest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   month          517 non-null    object 
 1   day            517 non-null    object 
 2   FFMC           517 non-null    float64
 3   DMC            517 non-null    float64
 4   DC             517 non-null    float64
 5   ISI            517 non-null    float64
 6   temp           517 non-null    float64
 7   RH             517 non-null    int64  
 8   wind           517 non-null    float64
 9   rain           517 non-null    float64
 10  area           517 non-null    float64
 11  dayfri         517 non-null    int64  
 12  daymon         517 non-null    int64  
 13  daysat         517 non-null    int64  
 14  daysun         517 non-null    int64  
 15  daythu         517 non-null    int64  
 16  daytue         517 non-null    int64  
 17  daywed         517 non-null    int64  
 18  monthapr  

In [11]:
forest[forest.duplicated()].shape

(8, 31)

In [12]:
forest[forest.duplicated()]

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
53,aug,wed,92.1,111.2,654.1,9.6,20.4,42,4.9,0.0,...,0,0,0,0,0,0,0,0,0,small
100,aug,sun,91.4,142.4,601.4,10.6,19.8,39,5.4,0.0,...,0,0,0,0,0,0,0,0,0,small
215,mar,sat,91.7,35.8,80.8,7.8,17.0,27,4.9,0.0,...,0,0,0,0,1,0,0,0,0,large
303,jun,fri,91.1,94.1,232.1,7.1,19.2,38,4.5,0.0,...,0,0,0,1,0,0,0,0,0,small
426,aug,thu,91.6,248.4,753.8,6.3,20.4,56,2.2,0.0,...,0,0,0,0,0,0,0,0,0,small
461,aug,sat,93.7,231.1,715.1,8.4,18.9,64,4.9,0.0,...,0,0,0,0,0,0,0,0,0,small
501,aug,tue,96.1,181.1,671.2,14.3,21.6,65,4.9,0.8,...,0,0,0,0,0,0,0,0,0,small
508,aug,fri,91.0,166.9,752.6,7.1,25.9,41,3.6,0.0,...,0,0,0,0,0,0,0,0,0,small


In [ ]:
# not duplicates

In [15]:
label_encode = LabelEncoder()

forest['month'] = label_encode.fit_transform(forest['month'])
forest['day'] = label_encode.fit_transform(forest['day'])
forest['size_category'] = label_encode.fit_transform(forest['size_category'])

In [16]:
forest.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,7,0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,1
1,10,5,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,1
2,10,2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,1
3,7,0,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,1
4,7,3,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,1


In [17]:
x = forest.iloc[:,0:-1]
y = forest.iloc[:,-1]

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=8)

## SVM - Linear Model

In [18]:
# creating the model
lin_model = SVC(kernel='linear')

#fiting the model
lin_model.fit(x_train,y_train)

# predicting on training Data
pred_train_linear = lin_model.predict(x_train)

# Accuracy & Confusion Matix over the Training data

print(f'The Accuracy of Linear Model over Training data is = {accuracy_score(y_train,pred_train_linear)*100}%')

print(f'Confusion Matrix is \n {confusion_matrix(y_train,pred_train_linear)}')


The Accuracy of Linear Model over Training data is = 100.0%
Confusion Matrix is 
 [[ 98   0]
 [  0 263]]


In [19]:
# predicting on testing Data
pred_test_linear = lin_model.predict(x_test)

# Accuracy & Confusion Matix over the Testing data

print(f'The Accuracy of Linear Model over Testing data is = {accuracy_score(y_test,pred_test_linear)*100}%')

print(f'Confusion Matrix is \n {confusion_matrix(y_test,pred_test_linear)}')


The Accuracy of Linear Model over Testing data is = 99.35897435897436%
Confusion Matrix is 
 [[ 40   1]
 [  0 115]]


## SVM - Polynomial

In [20]:
# creating the model
poly_model = SVC(kernel='poly')

#fiting the model
poly_model.fit(x_train,y_train)

# predicting on training Data
pred_train_poly = poly_model.predict(x_train)

# Accuracy & Confusion Matix over the Training data

print(f'The Accuracy of Polynomial Model over Training data is = {accuracy_score(y_train,pred_train_poly)*100}%')

print(f'Confusion Matrix is \n {confusion_matrix(y_train,pred_train_poly)}')


The Accuracy of Polynomial Model over Training data is = 76.45429362880887%
Confusion Matrix is 
 [[ 13  85]
 [  0 263]]


In [21]:
# predicting on testing Data
pred_test_poly = poly_model.predict(x_test)

# Accuracy & Confusion Matix over the Testing data

print(f'The Accuracy of Polynomial Model over Testing data is = {accuracy_score(y_test,pred_test_poly)*100}%')

print(f'Confusion Matrix is \n {confusion_matrix(y_test,pred_test_poly)}')


The Accuracy of Polynomial Model over Testing data is = 77.56410256410257%
Confusion Matrix is 
 [[  6  35]
 [  0 115]]


## RBF - Exponential

In [24]:
# finding optimal no of dimension

rbf_model = SVC()

param = [{'kernel':['rbf'],'C':[20, 15, 12, 11, 10,5, 0.1],'gamma':[5, 50, 10, 0.5]}]

grid = GridSearchCV(estimator=rbf_model,param_grid=param)

grid.fit(x_train,y_train)

print(f'The best Score is {np.round(grid.best_score_,4)}')
print(f'The Best Parameter for RBF is = {grid.best_params_}')

The best Score is 0.7396
The Best Parameter for RBF is = {'C': 20, 'gamma': 0.5, 'kernel': 'rbf'}


In [25]:
# Model of Expontial form - e^-x
svm_rbf_model = SVC(kernel='rbf',C =20, gamma=0.5)

# fitting the Data
svm_rbf_model.fit(x_train,y_train)

# prediction on train Data
pred_rbf = svm_rbf_model.predict(x_train)

# Accuracy & Confusion Matix over the Training data

print(f'The Accuracy over Training data is = {accuracy_score(y_train,pred_rbf)*100}%')

print(f'Confusion Matrix is \n {confusion_matrix(y_train,pred_rbf)}')

The Accuracy over Training data is = 100.0%
Confusion Matrix is 
 [[ 98   0]
 [  0 263]]


In [26]:
# prediction on testing Data
pred_rbf_test = svm_rbf_model.predict(x_test)

# Accuracy & Confusion Matix over the Testing data

print(f'The Accuracy over Testing data is = {accuracy_score(y_test,pred_rbf_test)*100}%')

print(f'Confusion Matrix is \n {confusion_matrix(y_test,pred_rbf_test)}')

The Accuracy over Testing data is = 74.35897435897436%
Confusion Matrix is 
 [[  1  40]
 [  0 115]]


# without considering the Categorical data

In [30]:
X = forest.iloc[:,2:-1]
Y = forest.iloc[:,-1]
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=8)

### Linear Model

In [31]:
# creating the model
lin_model = SVC(kernel='linear')

#fiting the model
lin_model.fit(X_train,Y_train)

# predicting on training Data
pred_train_linear = lin_model.predict(X_train)

# Accuracy & Confusion Matix over the Training data

print(f'The Accuracy of Linear Model over Training data is = {accuracy_score(Y_train,pred_train_linear)*100}%')

print(f'Confusion Matrix is \n {confusion_matrix(Y_train,pred_train_linear)}')


The Accuracy of Linear Model over Training data is = 100.0%
Confusion Matrix is 
 [[ 98   0]
 [  0 263]]


In [32]:
# predicting on testing Data
pred_test_linear = lin_model.predict(X_test)

# Accuracy & Confusion Matix over the Testing data

print(f'The Accuracy of Linear Model over Testing data is = {accuracy_score(Y_test,pred_test_linear)*100}%')

print(f'Confusion Matrix is \n {confusion_matrix(Y_test,pred_test_linear)}')

The Accuracy of Linear Model over Testing data is = 98.71794871794873%
Confusion Matrix is 
 [[ 40   1]
 [  1 114]]


### Polynomial

In [33]:
# creating the model
poly_model = SVC(kernel='poly')

#fiting the model
poly_model.fit(X_train,Y_train)

# predicting on training Data
pred_train_poly = poly_model.predict(X_train)

# Accuracy & Confusion Matix over the Training data

print(f'The Accuracy of Polynomial Model over Training data is = {accuracy_score(Y_train,pred_train_poly)*100}%')

print(f'Confusion Matrix is \n {confusion_matrix(Y_train,pred_train_poly)}')


The Accuracy of Polynomial Model over Training data is = 76.45429362880887%
Confusion Matrix is 
 [[ 13  85]
 [  0 263]]


In [34]:
# predicting on testing Data
pred_test_poly = poly_model.predict(X_test)

# Accuracy & Confusion Matix over the Testing data

print(f'The Accuracy of Polynomial Model over Testing data is = {accuracy_score(Y_test,pred_test_poly)*100}%')

print(f'Confusion Matrix is \n {confusion_matrix(Y_test,pred_test_poly)}')


The Accuracy of Polynomial Model over Testing data is = 77.56410256410257%
Confusion Matrix is 
 [[  6  35]
 [  0 115]]


### Exponential - RBF

In [35]:
# finding optimal no of dimension
rbf_model = SVC()

param = [{'kernel':['rbf'],'C':[20, 15, 12, 11, 10,5, 0.1],'gamma':[5, 50, 10, 0.5]}]

grid = GridSearchCV(estimator=rbf_model,param_grid=param)

grid.fit(X_train,Y_train)

print(f'The best Score is {np.round(grid.best_score_,4)}')
print(f'The Best Parameter for RBF is = {grid.best_params_}')

The best Score is 0.7396
The Best Parameter for RBF is = {'C': 20, 'gamma': 0.5, 'kernel': 'rbf'}


In [36]:
# Model of Expontial form - e^-x
svm_rbf_model = SVC(kernel='rbf',C =20, gamma=0.5)

# fitting the Data
svm_rbf_model.fit(X_train,Y_train)

# prediction on train Data
pred_rbf = svm_rbf_model.predict(X_train)

# Accuracy & Confusion Matix over the Training data

print(f'The Accuracy over Training data is = {accuracy_score(Y_train,pred_rbf)*100}%')

print(f'Confusion Matrix is \n {confusion_matrix(Y_train,pred_rbf)}')

The Accuracy over Training data is = 100.0%
Confusion Matrix is 
 [[ 98   0]
 [  0 263]]


In [37]:
# prediction on testing Data
pred_rbf_test = svm_rbf_model.predict(X_test)

# Accuracy & Confusion Matix over the Testing data

print(f'The Accuracy over Testing data is = {accuracy_score(Y_test,pred_rbf_test)*100}%')

print(f'Confusion Matrix is \n {confusion_matrix(Y_test,pred_rbf_test)}')

The Accuracy over Testing data is = 74.35897435897436%
Confusion Matrix is 
 [[  1  40]
 [  0 115]]
